In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from math import pi
from datetime import datetime  as dt
import matplotlib
import networkx as nx

# Import API key
from api_keys import api_key

In [2]:
# Dependencies
# ----------------------------------
# Imports the method used for connecting to DBs
from sqlalchemy import create_engine

# Imports the methods needed to abstract classes into tables
from sqlalchemy.ext.declarative import declarative_base

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float 

In [3]:
# Create City/location and transport Classes
# ----------------------------------

# Sets an object to utilize the default declarative base in SQL Alchemy
Base = declarative_base()


# Creates Classes which will serve as the anchor points for our Tables
class State(Base):
    __tablename__ = 'location'
    id = Column(Integer, primary_key=True)
    city_name = Column(String(255))
    date = Column(String(255))
    cases = Column(Integer)
    deaths = Column(Integer)
    new_cases = Column(Integer)
    new_deaths = Column(Integer)

In [4]:
covid_cases = "all_state_csv.csv"
covid_mobility = "google_mobility/us-mobility.csv"

In [5]:
cases_df = pd.read_csv(covid_cases)
mobility_df = pd.read_csv(covid_mobility)

In [6]:
mobility_df

,date,state,region,retail_recreation,grocery_pharmacy,parks,transit_stations,workplaces,residential
0,2020-03-29,Alabama,Total,-41.0,-13.0,19.0,-30.0,-32.0,9.0
1,2020-03-29,Alabama,Autauga County,-42.0,-8.0,-14.0,NaN,-35.0,16.0
2,2020-03-29,Alabama,Baldwin County,-47.0,-19.0,-27.0,-24.0,-32.0,9.0
3,2020-03-29,Alabama,Barbour County,-27.0,-20.0,NaN,NaN,-23.0,NaN
4,2020-03-29,Alabama,Bibb County,-25.0,-10.0,NaN,NaN,-32.0,NaN
...,...,...,...,...,...,...,...,...,...
2855,2020-03-29,Wyoming,Sweetwater County,-33.0,-9.0,NaN,6.0,-22.0,15.0
2856,2020-03-29,Wyoming,Teton County,-83.0,-50.0,-57.0,-80.0,-70.0,NaN
2857,2020-03-29,Wyoming,Uinta County,-29.0,2.0,NaN,1.0,-18.0,NaN
2858,2020-03-29,Wyoming,Washakie County,-17.0,-7.0,NaN,NaN,-32.0,NaN


In [7]:
cases_df

,index_row,date,state,fips,cases,deaths,new_cases,new_deaths
0,0,2020-01-21,Washington,53,1,0,1,0
1,1,2020-01-22,Washington,53,1,0,0,0
2,2,2020-01-23,Washington,53,1,0,0,0
3,3,2020-01-24,Washington,53,1,0,0,0
4,4,2020-01-25,Washington,53,1,0,0,0
...,...,...,...,...,...,...,...,...
1824,1824,2020-03-31,Northern Mariana Islands,69,2,0,0,0
1825,1825,2020-04-01,Northern Mariana Islands,69,6,1,4,1
1826,1826,2020-04-02,Northern Mariana Islands,69,8,1,2,0
1827,1827,2020-04-03,Northern Mariana Islands,69,8,1,0,0


In [8]:
state_list = cases_df['state'].unique()

In [21]:
#for state in state_list:
df1 = cases_df.loc[cases_df['state'] == 'Alabama']
df2 = mobility_df.loc[mobility_df['state'] == 'Alabama']
#df3 = df1.join(df2,on='date',how='inner')
df3 = pd.merge(df1, df2, on='date')

In [22]:
df3

,index_row,date,state_x,fips,cases,deaths,new_cases,new_deaths,state_y,region,retail_recreation,grocery_pharmacy,parks,transit_stations,workplaces,residential
0,1683,2020-03-29,Alabama,1,830,5,110,1,Alabama,Total,-41.0,-13.0,19.0,-30.0,-32.0,9.0
1,1683,2020-03-29,Alabama,1,830,5,110,1,Alabama,Autauga County,-42.0,-8.0,-14.0,NaN,-35.0,16.0
2,1683,2020-03-29,Alabama,1,830,5,110,1,Alabama,Baldwin County,-47.0,-19.0,-27.0,-24.0,-32.0,9.0
3,1683,2020-03-29,Alabama,1,830,5,110,1,Alabama,Barbour County,-27.0,-20.0,NaN,NaN,-23.0,NaN
4,1683,2020-03-29,Alabama,1,830,5,110,1,Alabama,Bibb County,-25.0,-10.0,NaN,NaN,-32.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,1683,2020-03-29,Alabama,1,830,5,110,1,Alabama,Tuscaloosa County,-46.0,-14.0,-1.0,-29.0,-40.0,12.0
64,1683,2020-03-29,Alabama,1,830,5,110,1,Alabama,Walker County,-37.0,-10.0,NaN,-32.0,-24.0,12.0
65,1683,2020-03-29,Alabama,1,830,5,110,1,Alabama,Washington County,NaN,-13.0,NaN,NaN,-25.0,NaN
66,1683,2020-03-29,Alabama,1,830,5,110,1,Alabama,Wilcox County,-49.0,-17.0,NaN,NaN,-31.0,NaN
